##### Packages

In [7]:
import numpy as np
import pandas as pd
from numba import jit
import random
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import display
from tabulate import tabulate
import warnings
import torch
import random
import torch.nn as nn
import torch.optim as optim
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10,8) 
plt.style.use('ggplot')

##### Setup

In [8]:
def gametype_to_ran(gametype = '1236'):
    # convert game type into upper bounds for token values
    R1 = 3 ** int(gametype[0]) - 1
    R2 = 3 ** int(gametype[1]) - 1
    R3 = 3 ** int(gametype[2]) - 1
    R4 = 3 ** int(gametype[3]) - 1
    return R1, R2, R3, R4

In [9]:
def token_init(ntokens, nbuyers, nsellers, R1, R2, R3, R4):
    # generate values
    A = np.random.uniform(0, R1)  # common to all
    B = np.random.uniform(0, R2, (2, ))  # distinct for buyers and sellers
    C = np.random.uniform(0, R3, (2, ntokens))
    D = np.random.uniform(0, R4, (ntokens, nbuyers+nsellers))
    E = D + A
    
    # buyer valuations
    redemption_values = E[0:nbuyers]+C[0,:].reshape(-1,1)+B[0]
    sorted_indices = np.argsort(redemption_values, axis=1)[:, ::-1]
    redemption_values = np.take_along_axis(redemption_values, sorted_indices, axis=1)

    # seller costs
    token_costs = E[nbuyers:nbuyers+nsellers]+C[1,:].reshape(-1,1)+B[1]
    sorted_indices_costs = np.argsort(token_costs, axis=1)
    token_costs = np.take_along_axis(token_costs, sorted_indices_costs, axis=1)
    return np.round(redemption_values, 1), np.round(token_costs, 1)

In [10]:
def compute_demand_supply(redemption_values,token_costs,nbuyers,ntokens,granularity=100):
    max_price = np.max(redemption_values)
    min_price = np.min(token_costs)
    P_grid = np.linspace(min_price,max_price,granularity)
    demand_schedule = np.zeros((granularity),dtype = 'int')
    supply_schedule = np.zeros((granularity), dtype = 'int')
    for i, p in enumerate(P_grid):
        demand_schedule[i] = np.sum(p<=redemption_values)  
        supply_schedule[i] = np.sum(p>=token_costs) 
    return demand_schedule, supply_schedule, P_grid, min_price, max_price

In [11]:
def equilibrium(demand_schedule,supply_schedule,P_grid):
    p_eqbm, q_eqbm = [], np.nan
    for i, p in enumerate(P_grid):
        if demand_schedule[i] == supply_schedule[i]: # when sellers are ready to sell
            p_eqbm.append(p)
            q_eqbm = demand_schedule[i] 
    return np.nanmean(p_eqbm), q_eqbm

In [12]:
def graph(demand_schedule, supply_schedule, P_grid, min_price, max_price, p_eqbm, q_eqbm, 
                        period_bids, period_asks, period_prices, period_sales, 
                        redemption_values, token_costs, ntokens, nbuyers, nsellers, nsteps):
    plt.plot(demand_schedule, P_grid, label = 'Demand Curve')
    plt.plot(supply_schedule, P_grid, label = 'Supply Curve')
    plt.plot(period_prices, color='green', linestyle='--', label='Mean Real Prices')
    plt.axhline(y=np.nanmean(p_eqbm), color='black', linestyle='--', label='Mean Eqbm Prices')
    prices = []
    print(len(period_prices))
    for i in range(nsteps):
        if (period_prices[i] > 0):
            plt.scatter([period_sales[i]]*len(period_bids[i]), period_bids[i], s = 10, marker = 'x', c = 'red')
            plt.scatter([period_sales[i]]*len(period_asks[i]), period_asks[i], s = 10, marker = 'o', c = 'blue')
        else:
            pass  
    plt.legend()
    text_content = f'q*={q_eqbm}, mean(q)={np.round(np.nanmax(period_sales),1)},p*={np.round(p_eqbm,1)}, mean(p)={np.round(np.nanmean(period_prices),1)}'
    plt.title(text_content)
    plt.show()
    
    plt.plot(period_bids, c = 'r', linestyle='--')
    plt.plot(period_asks, c = 'b', linestyle='--')
    plt.scatter(range(nsteps), period_prices, c = 'g')
    plt.title('Bids (red), Asks (blue) and Prices (green) over trading steps')
    plt.show()


In [13]:
def current_bid_ask(bids, asks):
    if np.all(np.isnan(bids)) == False:
        current_bid_idx = np.nanargmax(bids)
        current_bid = np.nanmax(bids)
    else:
        current_bid_idx = np.nan
        current_bid = np.nan

    if np.all(np.isnan(asks)) == False:
        current_ask_idx = np.nanargmin(asks)
        current_ask = np.nanmin(asks)
    else:
        current_ask_idx = np.nan
        current_ask = np.nan
    return current_ask, current_ask_idx, current_bid, current_bid_idx


def buy_sell(db, current_bid, current_bid_idx, current_ask, current_ask_idx):
        sale, price, bprofit, sprofit = 0, np.nan, 0, 0 
        buy = db.buyers[current_bid_idx].buy(current_bid,current_ask)
        sell = db.sellers[current_ask_idx].sell(current_bid,current_ask)
        try:
            if buy and not sell:
                price = current_ask
            elif sell and not buy:
                price = current_bid
            elif sell and buy: 
                #price = np.random.choice([current_bid,current_ask])
                price = (current_bid + current_ask)/2
            else:
                price = np.nan
                
            if price > 0:
                db.buyers[current_bid_idx].transact(price)
                db.sellers[current_ask_idx].transact(price)
                sale = 1
                bprofit = db.buyers[current_bid_idx].step_profit
                sprofit = db.sellers[current_ask_idx].step_profit
        except:
            buy = np.nan
            sell = np.nan
        return sale, price, bprofit, sprofit, buy, sell

In [14]:
def graph_period(db, rnd, period):
    period_bids = list(db.get_period(rnd, period).bids)
    period_asks = list(db.get_period(rnd, period).asks)
    period_prices = list(db.get_period(rnd, period).price)
    period_sales = np.cumsum(np.where(db.get_period(rnd, period).price > 0,1,0))
    [_, demand_schedule, supply_schedule, P_grid, min_price, max_price, redemption_values, token_costs, p_eqbm, q_eqbm] = db.get_round(rnd).iloc[0].tolist()
    graph(demand_schedule, supply_schedule, P_grid, min_price, max_price, p_eqbm, q_eqbm, period_bids, period_asks, period_prices, period_sales, redemption_values, token_costs, db.ntokens, db.nbuyers, db.nsellers, db.nsteps)

In [15]:
class Database:
    def __init__(self, game_metadata, buyer_strategies, seller_strategies):
        self.nrounds, self.nperiods, self.ntokens, self.nbuyers, self.nsellers, self.nsteps, self.R1, self.R2, self.R3, self.R4 = game_metadata
        self.step_data = pd.DataFrame(columns=['rnd', 'period', 'step', 'bids','asks','current_bid','current_bid_idx','current_ask','current_ask_idx','buy','sell','price','sale', 'bprofit', 'sprofit'])
        self.round_data = pd.DataFrame(columns=['rnd', 'demand_schedule', 'supply_schedule', 'P_grid', 'redemption_values', 'token_costs','p_eqbm', 'q_eqbm'])
        self.buyers, self.sellers = generate_agents(buyer_strategies, seller_strategies, game_metadata)
        
    def add_step(self, data):
        self.step_data.loc[len(self.step_data.index)] = data
          
    def get_period(self, rnd, period):
        temp = self.step_data[(self.step_data.rnd==rnd) & (self.step_data.period==period)]
        temp = temp[['step', 'bids', 'asks', 'current_bid', 'current_bid_idx', 'current_ask','current_ask_idx','buy','sell','price', 'bprofit', 'sprofit']]
        return temp
    
    def reset_period(self, rnd):
        redemption_values, token_costs = self.get_round(rnd)[['redemption_values', 'token_costs']]
        for i in range(self.nbuyers):
            self.buyers[i].reset(self, rnd)
        for i in range(self.nsellers):
            self.sellers[i].reset(self, rnd)

    def reset_round(self):
        redemption_values, token_costs = token_init(ntokens, nbuyers, nsellers, R1, R2, R3, R4)
        demand_schedule, supply_schedule, P_grid, min_price, max_price = compute_demand_supply(redemption_values,token_costs, self.nbuyers, self.ntokens)
        p_eqbm, q_eqbm = equilibrium(demand_schedule,supply_schedule,P_grid) 
        self.round_data.loc[len(self.round_data.index)] = [rnd, demand_schedule, supply_schedule, P_grid, redemption_values, token_costs, p_eqbm, q_eqbm]
      
    def get_round(self, rnd):
        temp = self.round_data[(self.round_data.rnd==rnd)]
        temp = temp[['rnd', 'demand_schedule', 'supply_schedule', 'P_grid', 'redemption_values', 'token_costs','p_eqbm', 'q_eqbm']]
        return temp


In [16]:
class Trader:
    def __init__(self, game_metadata, name, buyer=True):
        self.name, self.buyer, self.index = name, buyer, int(name[1])
        self.nrounds, self.nperiods, self.ntokens, self.nbuyers, self.nsellers, self.nsteps, self.R1, self.R2, self.R3, self.R4, = game_metadata 

    def reset(self, db, rnd):
        round_data = db.get_round(rnd)
        self.num_tokens_traded = 0
        if self.buyer == True:
            self.token_values = list(round_data.redemption_values.item()[self.index])
        else:
            self.token_values = list(round_data.token_costs.item()[self.index])

    def transact(self, price):
        self.num_tokens_traded += 1
        if self.buyer == True:
            self.step_profit = self.value-price
        else:
            self.step_profit = price-self.value
    
    def next_token(self):
        if self.num_tokens_traded == self.ntokens:
            self.value = np.nan
        else:
            self.value = self.token_values[self.num_tokens_traded]

    def buy(self, current_bid, current_ask):
        if self.value >= current_ask:
            return True
        else:
            return False
        
    def sell(self, current_bid, current_ask):
        if current_bid >= self.value:
            return True
        else:
            return False

In [17]:
class Honest(Trader):
    def __init__(self, game_metadata, name, buyer=True):
        super().__init__(game_metadata, name, buyer)

    def bid(self, db):
        self.next_token()
        self.bid_amount = self.value
        return self.bid_amount
    
    def ask(self, db):
        self.next_token()
        self.ask_amount = self.value
        return self.ask_amount

In [18]:

def generate_agents(buyer_strategies, seller_strategies, game_metadata):
    buyers = []
    for idx,i in enumerate(buyer_strategies):
        if i == 'Honest':
            buyers.append(Honest(game_metadata,'B'+str(idx),buyer=True))    
    sellers = []
    for idx,i in enumerate(seller_strategies):
        if i == 'Honest':
            sellers.append(Honest(game_metadata,'S'+str(idx),buyer=False))     
    return buyers, sellers



In [74]:
buyer_strategies, seller_strategies = ['Honest', 'Honest',  'Honest',  'Honest'], ['Honest', 'Honest',  'Honest',  'Honest']
nbuyers, nsellers = len(buyer_strategies), len(seller_strategies)
nrounds, nperiods, ntokens, nsteps, gametype, nbuyers, nsellers = 1, 2, 4, 25, '1234', len(buyer_strategies), len(seller_strategies)
R1, R2, R3, R4 = gametype_to_ran(gametype)
game_metadata = [nrounds, nperiods, ntokens, nbuyers, nsellers, nsteps, R1, R2, R3, R4]
db = Database(game_metadata, buyer_strategies, buyer_strategies)

for rnd in range(nrounds):
    db.reset_round()
    for period in range(nperiods):
        db.reset_period(rnd)
        for step in range(nsteps):
            bids = [buyer.bid(db) for buyer in db.buyers]
            asks = [seller.ask(db) for seller in db.sellers]
            current_ask, current_ask_idx, current_bid, current_bid_idx = current_bid_ask(bids, asks) 
            sale, price, bprofit, sprofit, buy, sell = buy_sell(db, current_bid, current_bid_idx, current_ask, current_ask_idx)
            step_data = [rnd,period,step,bids,asks,current_bid,current_bid_idx,current_ask,current_ask_idx,buy,sell,price,sale,bprofit,sprofit]
            db.add_step(step_data)
            
db.get_period(0, 0).head(50)

(4, 8) (4,)
4
(0, 8) (4,)


ValueError: operands could not be broadcast together with shapes (0,8) (4,1) 

In [55]:
[_, demand_schedule, supply_schedule, P_grid, redemption_values, token_costs, p_eqbm, q_eqbm] = db.get_round(0).iloc[0].tolist()

IndexError: single positional indexer is out-of-bounds

In [34]:
P_grid

array([28.8       , 29.35151515, 29.9030303 , 30.45454545, 31.00606061,
       31.55757576, 32.10909091, 32.66060606, 33.21212121, 33.76363636,
       34.31515152, 34.86666667, 35.41818182, 35.96969697, 36.52121212,
       37.07272727, 37.62424242, 38.17575758, 38.72727273, 39.27878788,
       39.83030303, 40.38181818, 40.93333333, 41.48484848, 42.03636364,
       42.58787879, 43.13939394, 43.69090909, 44.24242424, 44.79393939,
       45.34545455, 45.8969697 , 46.44848485, 47.        , 47.55151515,
       48.1030303 , 48.65454545, 49.20606061, 49.75757576, 50.30909091,
       50.86060606, 51.41212121, 51.96363636, 52.51515152, 53.06666667,
       53.61818182, 54.16969697, 54.72121212, 55.27272727, 55.82424242,
       56.37575758, 56.92727273, 57.47878788, 58.03030303, 58.58181818,
       59.13333333, 59.68484848, 60.23636364, 60.78787879, 61.33939394,
       61.89090909, 62.44242424, 62.99393939, 63.54545455, 64.0969697 ,
       64.64848485, 65.2       , 65.75151515, 66.3030303 , 66.85

In [26]:
def graph_period(db, rnd, period):
    period_bids = list(db.get_period(rnd, period).bids)
    period_asks = list(db.get_period(rnd, period).asks)
    period_prices = list(db.get_period(rnd, period).price)
    period_sales = np.cumsum(np.where(db.get_period(rnd, period).price > 0,1,0))
    [_, demand_schedule, supply_schedule, P_grid, redemption_values, token_costs, p_eqbm, q_eqbm] = db.get_round(rnd).iloc[0].tolist()
    graph(demand_schedule, supply_schedule, P_grid, p_eqbm, q_eqbm, period_bids, period_asks, period_prices, period_sales, redemption_values, token_costs, db.ntokens, db.nbuyers, db.nsellers, db.nsteps)

In [27]:
def graph(demand_schedule, supply_schedule, P_grid, p_eqbm, q_eqbm, period_bids, period_asks, 
          period_prices, period_sales, redemption_values, token_costs, ntokens, nbuyers, nsellers, nsteps):
    plt.plot(demand_schedule, P_grid, label = 'Demand Curve')
    plt.plot(supply_schedule, P_grid, label = 'Supply Curve')
    plt.plot(period_prices, color='green', linestyle='--', label='Mean Real Prices')
    plt.axhline(y=np.nanmean(p_eqbm), color='black', linestyle='--', label='Mean Eqbm Prices')
    prices = []
    print(len(period_prices), nsteps, nbuyers, nsellers)
    for i in range(nsteps):
        if (period_prices[i] > 0):
            plt.scatter([period_sales[i]]*len(period_bids[i]), period_bids[i], s = 10, marker = 'x', c = 'red')
            plt.scatter([period_sales[i]]*len(period_asks[i]), period_asks[i], s = 10, marker = 'o', c = 'blue')
        else:
            pass  
    plt.legend()
    text_content = f'q*={q_eqbm}, mean(q)={np.round(np.nanmax(period_sales),1)},p*={np.round(p_eqbm,1)}, mean(p)={np.round(np.nanmean(period_prices),1)}'
    plt.title(text_content)
    plt.show()
    
    plt.plot(period_bids, c = 'r', linestyle='--')
    plt.plot(period_asks, c = 'b', linestyle='--')
    plt.scatter(range(nsteps), period_prices, c = 'g')
    plt.title('Bids (red), Asks (blue) and Prices (green) over trading steps')
    plt.show()



In [28]:
graph_period(db, rnd, period)

NameError: name 'db' is not defined

In [29]:
nepisodes = 10000
bprofit_history = []
for episode in range(nepisodes):

    for idx,i in enumerate(range(nbuyers)):
        db.buyers[i].reset(token_values[i])
    for idx,i in enumerate(range(nbuyers,nbuyers+nsellers)):
        db.sellers[idx].reset(token_values[i])

    for step in range(nsteps):  
        bids = [buyer.bid(db) for buyer in db.buyers]
        asks = [seller.ask(db) for seller in db.sellers]
        current_ask, current_ask_idx, current_bid, current_bid_idx = current_bid_ask(bids, asks) 
        sale, price, bprofit, sprofit, buy, sell = buy_sell(db, current_bid, current_bid_idx, current_ask, current_ask_idx)
        step_data = [rnd,period,step,bids,asks,current_bid,current_bid_idx,current_ask,current_ask_idx,buy,sell,price,sale,bprofit,sprofit, p_eqbm, q_eqbm]
        db.add_step(step_data)
        #print(db.buyers[0].bid_amount, db.buyers[0].value, current_bid_idx)

        # observe reward
        reward = 0
        if (current_bid_idx == 0) and (sale == 1):
            reward = bprofit
        bprofit_history.append(reward)
            
        db.buyers[0].observe_reward(reward)
        
        # train
        db.buyers[0].train()

    # observe
    if episode % (nsteps*10) == 0:
        print(episode, np.mean(bprofit_history[-nsteps:]), db.buyers[0].epsilon)
        print(db.get_period(rnd, period).tail(10)[['step','current_bid','current_ask','price','bprofit']])

NameError: name 'nbuyers' is not defined

In [30]:

def generate_agents(buyer_strategies, seller_strategies, game_metadata):
    buyers = []
    for idx,i in enumerate(buyer_strategies):
        if i == 'Honest':
            buyers.append(Honest(game_metadata,'B'+str(idx),buyer=True))    
    sellers = []
    for idx,i in enumerate(seller_strategies):
        if i == 'Honest':
            sellers.append(Honest(game_metadata,'S'+str(idx),buyer=False))     
    return buyers, sellers


##### Equilibrium

##### Plotting

In [31]:
def plot_steps(df, token_values, ntokens, nbuyers, nsellers, nsteps, R1, R2, R3, R4):
    period_bids = list(df.bids)
    period_asks = list(df.asks)
    period_prices = list(df.price)
    period_sales = np.cumsum(np.where(df.price > 0,1,0))
    ntokens, nbuyers, nsellers, nsteps, R1, R2, R3, R4 = ntokens, nbuyers, nsellers, nsteps, R1, R2, R3, R4
    plot_period_results(period_bids, period_asks, period_prices, period_sales, token_values, ntokens, nbuyers, nsellers, nsteps, R1, R2, R3, R4)

##### Database

##### Outcomes

In [32]:
def individual_outcomes(db, buyers = True):
    df = db.step_data
    a = df.groupby(['current_ask_idx']).mean()[['current_ask', 'price','sprofit']]
    b = df.groupby(['current_ask_idx']).sum()[['sell','sprofit']]
    seller_outcome = pd.concat([a,b], axis = 1)
    seller_outcome.columns = ['mean_asks', 'mean_price', 'mean_profit','total_sales','total_profit']
    a = df.groupby(['current_bid_idx']).mean()[['current_bid', 'price','bprofit']]
    b = df.groupby(['current_bid_idx']).sum()[['sell','bprofit']]
    buyer_outcome = pd.concat([a,b], axis = 1)
    buyer_outcome.columns = ['mean_bids', 'mean_price', 'mean_profit','total_sales','total_profit']
    if buyers == True:
        return buyer_outcome
    else:
        return seller_outcome
    
def market_outcomes(db):
    df = db.step_data
    a = df.groupby(['rnd', 'period']).mean()[['price','p_eqbm','q_eqbm']]
    b = df.groupby(['rnd', 'period']).sum()[['sell']]
    market = pd.concat([a,b], axis = 1)
    market.columns = ['mean_market_price', 'p_eqbm', 'q_eqbm','total_sales']
    return market

##### Trading

In [33]:
class Trader:
    def __init__(self, game_metadata, name, buyer=True):
        self.name, self.buyer = name, buyer
        self.step_profit, self.bids_on_this_token = 0, 0, 0, 0, 0
        self.nrounds, self.nperiods, self.ntokens, self.nbuyers, self.nsellers, self.R1, self.R2, self.R3, self.R4, self.minprice, self.maxprice, self.nsteps = game_metadata 
        
    def reset(self, token_values):
        self.step_profit = 0
        self.token_values = list(token_values)
        if self.buyer == True:
            self.num_tokens_held = 0 # tokens held
            self.num_tokens_traded = 0
            self.bids_on_this_token = 0
        else:
            self.num_tokens_held = len(token_values)-1
            self.num_tokens_traded = 0
            self.asks_on_this_token = 0

    def transact(self, price):
        self.num_tokens_traded += 1
        if self.buyer == True:
            self.num_tokens_held += 1
            self.step_profit = self.value-price
            self.bids_on_this_token = 0
        else:
            self.num_tokens_held -= 1
            self.step_profit = price-self.value
            self.asks_on_this_token = 0
    
    def next_token(self):
        if self.buyer == True:
            if self.num_tokens_held == self.ntokens:
                self.value = np.nan
            else:
                self.value = self.token_values[self.num_tokens_held]
        else:
            if self.num_tokens_held == 0:
                self.value = np.nan
            else:
                self.value = self.token_values[self.num_tokens_held]

    def buy(self, current_bid, current_ask):
        if self.value >= (current_bid + current_ask)/2:
            return True
        else:
            return False
        
    def sell(self, current_bid, current_ask):
        if (current_bid + current_ask)/2 >= self.value:
            return True
        else:
            return False

In [34]:
class QNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, output_size)
        
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        return self.fc2(x)

In [35]:
class DQN(Trader):
    def __init__(self, public_information, name, buyer=True):
        super().__init__(public_information, name, buyer)
        self.num_actions = 6
        self.num_states = 1
        self.alpha = 0.1
        self.gamma = 0
        self.initial_epsilon = 0.99
        self.epsilon = self.initial_epsilon
        self.epsilon_decay = 0.9999
        self.min_epsilon = 0.01
        self.q_network = QNetwork(self.num_states,self.num_actions)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=self.alpha)
        self.memory = []
        self.batch_size = 512
        self.action2bid = np.linspace(0, 1, self.num_actions)
        self.state = None
        
    def observe_state(self, db):
        self.next_token()
        try:
            data = db.step_data.iloc[-1]
            if data.step == db.nstep:
                self.state = [0]
            else:
                self.state = [data.step]
        except:
            self.state = [0]
        
    def bid(self, db):
        # find ranges for bidding
        self.next_token()
        self.max_bid = self.value
        self.min_bid = self.value*0.7
        
        # obtain state
        self.observe_state(db)
        state = torch.tensor(self.state, dtype=torch.float32)

        # take action from exploratory strategy
        if random.random() < self.epsilon:
            self.action = random.randint(0, self.num_actions-1)
        else:
            self.q_values = self.q_network(state)
            self.action = torch.argmax(self.q_values).item()
            
        # compute bid amount (how much shading)
        self.bid_fraction = self.action2bid[self.action]
        self.bid_amount = np.round(self.max_bid * self.bid_fraction + (1-self.bid_fraction) * self.min_bid,1)
        return self.bid_amount
        
    def observe_reward(self, reward):
        self.reward = reward
        self.memory.append((self.state, self.action, reward))
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)
    
    def train(self):        
        if len(self.memory) >= self.batch_size:
            batch = random.sample(self.memory, self.batch_size)
            states, actions, rewards = zip(*batch)
            states = torch.tensor(states, dtype=torch.float32)
            actions = torch.tensor(actions, dtype=torch.int64)
            rewards = torch.tensor(rewards, dtype=torch.float32)
            q_values = self.q_network(states.unsqueeze(1))
            q_values = q_values.squeeze(dim=1)
            target_q_values = rewards + self.gamma * torch.max(q_values, dim=1).values
            predicted_q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
            self.loss = nn.MSELoss()(predicted_q_values, target_q_values)
            self.optimizer.zero_grad()
            self.loss.backward()
            self.optimizer.step()

##### Q-Learning

In [47]:
buyer_strategies = ['DQN']
seller_strategies = ['Honest']
nrounds, nperiods, ntokens, R1, R2, R3, R4, minprice, maxprice, nsteps = 1, 1, 10, 20, 20, 20, 20, 1, 100, 10
nbuyers, nsellers = len(buyer_strategies), len(seller_strategies)
game_metadata = [nrounds, nperiods, ntokens, nbuyers, nsellers, R1, R2, R3, R4, minprice, maxprice, nsteps]
buyers, sellers = generate_agents(buyer_strategies, seller_strategies, game_metadata)
token_values = round_setup(ntokens, nbuyers, nsellers, R1, R2, R3, R4)
p_eqbm, q_eqbm = compute_demand_supply(token_values, ntokens, nbuyers,nsellers)
db = Database(game_metadata, buyers, sellers)

In [52]:
nepisodes = 10000
bprofit_history = []
for episode in range(nepisodes):

    for idx,i in enumerate(range(nbuyers)):
        db.buyers[i].reset(token_values[i])
    for idx,i in enumerate(range(nbuyers,nbuyers+nsellers)):
        db.sellers[idx].reset(token_values[i])

    for step in range(nsteps):  
        bids = [buyer.bid(db) for buyer in db.buyers]
        asks = [seller.ask(db) for seller in db.sellers]
        current_ask, current_ask_idx, current_bid, current_bid_idx = current_bid_ask(bids, asks) 
        sale, price, bprofit, sprofit, buy, sell = buy_sell(db, current_bid, current_bid_idx, current_ask, current_ask_idx)
        step_data = [rnd,period,step,bids,asks,current_bid,current_bid_idx,current_ask,current_ask_idx,buy,sell,price,sale,bprofit,sprofit, p_eqbm, q_eqbm]
        db.add_step(step_data)
        #print(db.buyers[0].bid_amount, db.buyers[0].value, current_bid_idx)

        # observe reward
        reward = 0
        if (current_bid_idx == 0) and (sale == 1):
            reward = bprofit
        bprofit_history.append(reward)
            
        db.buyers[0].observe_reward(reward)
        
        # train
        db.buyers[0].train()

    # observe
    if episode % (nsteps*10) == 0:
        print(episode, np.mean(bprofit_history[-nsteps:]), db.buyers[0].epsilon)
        print(db.get_period(rnd, period).tail(10)[['step','current_bid','current_ask','price','bprofit']])

0 1.6199999999999997 0.2660380220883367
       step  current_bid  current_ask  price  bprofit
13130     0         42.2         36.6   39.4      9.4
13131     1         40.7         39.9   40.3      6.8
13132     2         39.1         46.1    NaN      0.0
13133     3         39.1         46.1    NaN      0.0
13134     4         43.2         46.1    NaN      0.0
13135     5         40.5         46.1    NaN      0.0
13136     6         40.5         46.1    NaN      0.0
13137     7         39.8         46.1    NaN      0.0
13138     8         39.8         46.1    NaN      0.0
13139     9         39.8         46.1    NaN      0.0
100 1.6199999999999997 0.24071995332278487
       step  current_bid  current_ask  price  bprofit
14130     0         42.2         36.6   39.4      9.4
14131     1         40.7         39.9   40.3      6.8
14132     2         39.1         46.1    NaN      0.0
14133     3         39.1         46.1    NaN      0.0
14134     4         39.1         46.1    NaN      0.0

KeyboardInterrupt: 

In [58]:
rnd, period = 0,0
db.get_token_values(rnd).item()

ValueError: can only convert an array of size 1 to a Python scalar

In [60]:
db.buyers[0].token_values, db.sellers[0].token_values

([48.8, 47.1, 45.2, 43.7, 42.7, 40.5, 39.6, 36.8, 34.0, 31.9],
 [55.5, 55.0, 53.4, 52.4, 48.7, 48.6, 46.3, 46.1, 39.9, 36.6])

In [44]:
db.get_period(rnd, period).head(6)

,step,bids,asks,current_bid,current_bid_idx,current_ask,current_ask_idx,price,bprofit,sprofit,sales
0,0,[38.8],[44.2],38.8,0,44.2,0.0,NaN,0.0,0.0,0
1,1,[47.8],[44.2],47.8,0,44.2,0.0,46.0,4.0,1.8,1
2,2,[48.0],[46.8],48.0,0,46.8,0.0,47.4,2.1,0.6,2
3,0,[38.8],[44.2],38.8,0,44.2,0.0,NaN,0.0,0.0,2
4,1,[35.8],[44.2],35.8,0,44.2,0.0,NaN,0.0,0.0,2
5,2,[35.0],[44.2],35.0,0,44.2,0.0,NaN,0.0,0.0,2


In [45]:
db.get_period(rnd, period).tail(20)

,step,bids,asks,current_bid,current_bid_idx,current_ask,current_ask_idx,price,bprofit,sprofit,sales
24911,2,[44.8],[nan],44.8,0,NaN,NaN,NaN,0.00,0.00,12978
24912,0,[47.8],[44.2],47.8,0,44.2,0.0,46.00,4.00,1.80,12979
24913,1,[36.1],[46.8],36.1,0,46.8,0.0,NaN,0.00,0.00,12979
24914,2,[47.3],[46.8],47.3,0,46.8,0.0,47.05,2.45,0.25,12980
24915,0,[47.8],[44.2],47.8,0,44.2,0.0,46.00,4.00,1.80,12981
24916,1,[47.3],[46.8],47.3,0,46.8,0.0,47.05,2.45,0.25,12982
24917,2,[44.8],[nan],44.8,0,NaN,NaN,NaN,0.00,0.00,12982
24918,0,[36.5],[44.2],36.5,0,44.2,0.0,NaN,0.00,0.00,12982
24919,1,[38.0],[44.2],38.0,0,44.2,0.0,NaN,0.00,0.00,12982
24920,2,[47.8],[44.2],47.8,0,44.2,0.0,46.00,4.00,1.80,12983


In [71]:
db.get_period(rnd, period).iloc[600:606]

,step,bids,asks,current_bid,current_bid_idx,current_ask,current_ask_idx,price,bprofit,sprofit,sales
600,0,[29.7],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.1, 18.4]",29.7,0,13.1,6,21.40,18.40,8.30,601
601,1,[36.3],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",36.3,0,13.2,1,24.75,13.25,11.55,602
602,2,[27.7],"[37.9, 25.4, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",27.7,0,13.8,6,20.75,17.15,6.95,603
603,0,[27.9],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.1, 18.4]",27.9,0,13.1,6,20.50,19.30,7.40,604
604,1,[36.3],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",36.3,0,13.2,1,24.75,13.25,11.55,605
605,2,[35.1],"[37.9, 25.4, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",35.1,0,13.8,6,24.45,13.45,10.65,606


In [75]:
db.get_period(rnd, period).head(6)

,step,bids,asks,current_bid,current_bid_idx,current_ask,current_ask_idx,price,bprofit,sprofit,sales
0,0,[27.9],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.1, 18.4]",27.9,0,13.1,6,20.50,19.30,7.40,1
1,1,[37.4],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",37.4,0,13.2,1,25.30,12.70,12.10,2
2,2,[32.2],"[37.9, 25.4, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",32.2,0,13.8,6,23.00,14.90,9.20,3
3,0,[31.4],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.1, 18.4]",31.4,0,13.1,6,22.25,17.55,9.15,4
4,1,[38.0],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",38.0,0,13.2,1,25.60,12.40,12.40,5
5,2,[31.1],"[37.9, 25.4, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",31.1,0,13.8,6,22.45,15.45,8.65,6


In [77]:
db.get_period(rnd, period).tail(6)

,step,bids,asks,current_bid,current_bid_idx,current_ask,current_ask_idx,price,bprofit,sprofit,sales
23695,1,[26.6],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",26.6,0,13.2,1,19.90,18.10,6.70,23696
23696,2,[26.5],"[37.9, 25.4, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",26.5,0,13.8,6,20.15,17.75,6.35,23697
23697,0,[27.9],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.1, 18.4]",27.9,0,13.1,6,20.50,19.30,7.40,23698
23698,1,[32.3],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",32.3,0,13.2,1,22.75,15.25,9.55,23699
23699,2,[26.5],"[37.9, 25.4, 24.3, 29.2, 25.2, 25.4, 13.8, 18.4]",26.5,0,13.8,6,20.15,17.75,6.35,23700
23700,0,[27.9],"[37.9, 13.2, 24.3, 29.2, 25.2, 25.4, 13.1, 18.4]",27.9,0,13.1,6,20.50,19.30,7.40,23701
